# Extractive HuggingFace

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/qa-extractive-huggingface](https://github.com/huseinzol05/Malaya/tree/master/example/qa-extractive-huggingface).
    
</div>

<div class="alert alert-warning">

This module only trained on standard language structure, so it is not save to use it for local language structure.
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import logging

logging.basicConfig(level=logging.INFO)

In [3]:
%%time

import malaya
from pprint import pprint

CPU times: user 3.4 s, sys: 3.52 s, total: 6.92 s
Wall time: 2.54 s


### What is Extractive QA

is a reading comprehension problem, eg,

```json
{
    'title': 'Normans',
    'paragraphs': [
        {
            'context': 'Orang Norman (Norman: Nourmands; Perancis: Normands; Latin: Normanni) ialah orang-orang yang pada abad ke-10 dan ke-11 memberikan nama mereka kepada Normandy, sebuah wilayah di Perancis. Mereka diturunkan daripada Norse ("Norman" berasal daripada penyerang "Norseman") dan lanun dari Denmark, Iceland dan Norway yang, di bawah pimpinan mereka Rollo, bersetuju untuk bersumpah fealty kepada Raja Charles III dari Francia Barat. Melalui generasi asimilasi dan percampuran dengan penduduk asli Frankish dan Roman-Gaulish, keturunan mereka akan beransur-ansur bergabung dengan budaya Carolingian yang berpusat di Francia Barat. Identiti budaya dan etnik yang berbeza dari orang Norman muncul pada mulanya pada separuh pertama abad ke-10, dan ia terus berkembang pada abad-abad yang berjaya.',
            'qas': [
                {
                    'question': 'Di negara manakah Normandy berada?',
                    'answers': [
                        {'text': 'Perancis', 'answer_start': 177},
                        {'text': 'Perancis', 'answer_start': 177},
                        {'text': 'Perancis', 'answer_start': 177},
                        {'text': 'Perancis', 'answer_start': 177},
                    ],
                    'id': '56ddde6b9a695914005b9628',
                    'is_impossible': False,
                }
            ],
        }
    ],
}
```

So we need to give a long paragraph and multiple questions, and the model will return answers based on that paragraph with start and end spans. Read more about SQUAD dataset https://rajpurkar.github.io/SQuAD-explorer/

### List available HuggingFace models

In [4]:
malaya.qa.extractive.available_huggingface()

INFO:malaya.qa.extractive:tested on translated SQUAD V2 Dev set, https://github.com/huseinzol05/malay-dataset/tree/master/question-answer/squad
INFO:malaya.qa.extractive:tested on SQUAD V2 Dev set, https://rajpurkar.github.io/SQuAD-explorer/


,Size (MB),exact-ms,f1-ms,exact-en,f1-en
mesolitica/finetune-qa-t5-tiny-standard-bahasa-cased,139.0,0.426956,0.511303,0.469257,0.541063
mesolitica/finetune-qa-t5-small-standard-bahasa-cased,242.0,0.530340,0.616933,0.558784,0.629300
mesolitica/finetune-qa-t5-base-standard-bahasa-cased,892.0,0.570355,0.641370,0.570355,0.641370
mesolitica/finetune-extractive-qa-flan-t5-base,990.0,0.552198,0.624780,0.709291,0.779958


### Load HuggingFace model

In [5]:
model = malaya.qa.extractive.huggingface(model = 'mesolitica/finetune-qa-t5-small-standard-bahasa-cased')

In [6]:
model_flan = malaya.qa.extractive.huggingface(model = 'mesolitica/finetune-extractive-qa-flan-t5-base')

### Copy from wikipedia and news

In [7]:
# https://ms.wikipedia.org/wiki/Mohd_Najib_bin_Abdul_Razak

p_wikipedia = """
Najib razak telah dipilih untuk Parlimen Malaysia pada tahun 1976, 
pada usia 23 tahun, menggantikan bapanya duduk di kerusi Pekan yang berpangkalan di Pahang. 
Dari tahun 1982 hingga 1986 beliau menjadi Menteri Besar (Ketua Menteri) Pahang, 
sebelum memasuki persekutuan Kabinet Tun Dr Mahathir Mohamad pada tahun 1986 sebagai Menteri Kebudayaan, Belia dan Sukan. 
Beliau telah berkhidmat dalam pelbagai jawatan Kabinet sepanjang baki tahun 1980-an dan 1990-an, termasuk sebagai Menteri Pertahanan dan Menteri Pelajaran. 
Beliau menjadi Timbalan Perdana Menteri pada 7 Januari 2004, berkhidmat di bawah Perdana Menteri Tun Dato' Seri Abdullah Ahmad Badawi, 
sebelum menggantikan Badawi setahun selepas Barisan Nasional mengalami kerugian besar dalam pilihan raya 2008. 
Di bawah kepimpinan beliau, Barisan Nasional memenangi pilihan raya 2013, 
walaupun buat kali pertama dalam sejarah Malaysia pembangkang memenangi majoriti undi popular.
"""
q_wikipedia = [
    'pada umur berapa najib razak menggantikan ayahnya', 
    'Apakah jawatan yang pernah dipegang oleh Najib Razak',
    'what date najib become deputy prime minister',
]

In [8]:
# https://www.malaysiakini.com/news/574914

p_news = """
Bekas perdana menteri Najib Razak mempersoalkan tindakan polis yang menurutnya tidak serta-merta mengeluarkan kenyataan berhubung dakwaan Adun Perikatan Nasional (PN) "merancang" insiden rogol.
Sedangkan, kata ahli parlimen Pekan itu, polis pantas mengeluarkan kenyataan apabila dia dilapor terlupa mengimbas MySejahtera sebelum masuk restoran.
"Berita Najib lupa scan MySejahtera tular, kenyataan polis terus keluar. Berita Dr Mahathir Mohamad lupa scan, kenyataan, polis serta-merta keluar.
"Sebab itu saya pelik kenapa pihak polis belum sempat keluar apa-apa kenyataan berhubung kes seorang gadis membuat laporan polis untuk dakwa Adun PN rancang insiden rogolnya," katanya di Facebook hari ini.
Najib merujuk dakwaan seorang wanita yang mendakwa dirogol kenalan kepada Adun Gombak Setia, Hilman Idham.
Wanita itu mendakwa ahli politik dari Bersatu berkenaan merancang insiden yang berlaku pada 5 Dis lalu.
Menurut laporan polis pada 8 Mei, mangsa mendakwa kejadian itu berlaku di sebuah hotel di Selangor, yang pada masa itu berada di bawah perintah kawalan pergerakan bersyarat (PKPB).
"""

q_news = [
    'siapakah yang mempersoalkan tindakan polis', 
    'siapakah Adun Gombak Setia',
    'when does make the allegation',
]

In [9]:
# https://www.parlimen.gov.my/files/hindex/pdf/DR-19122022.pdf

p_parlimen = """
Menteri Dalam Negeri [Datuk Seri Saifuddin Nasution bin Ismail]:
Assalamualaikum warahmatullahi wabarakatuh. Tuan Yang diPertua, saya mohon
mencadangkan:
“Bahawa Majlis ini memerintahkan Ketua Polis Negara
menjaga supaya selama Penggal Dewan Rakyat Parlimen Kelima
Belas yang ada sekarang, jalan-jalan melalui lorong-lorong menuju ke
Dewan ini hendaklah sentiasa terbuka dan boleh dilalui dan jangan
ada apa-apa halangan menghalang Ahli-ahli hendak pergi dan balik
dari Dewan ini, jangan berlaku apa-apa kacau bilau di jalan-jalan
menuju ke Dewan ini, dan supaya jangan ada huru-hara di Bangunan
Dewan ini dan berhampiran dengan bangunan ini; dan Setiausaha
Dewan Rakyat hendaklah menyampaikan perintah ini kepada Ketua
Polis Negara yang tersebut itu”.
Terima kasih.
Timbalan Menteri Dalam Negeri [Datuk Seri Dr. Shamsul Anuar bin
Nasarah]: Tuan Yang di-Pertua, saya mohon menyokong.
Tuan Yang di-Pertua: Terima kasih. Ahli-ahli Yang Berhormat, masalahnya
ialah usul oleh Yang Berhormat Menteri Dalam Negeri yang telah dibacakan tadi
hendaklah dipersetujukan
"""

q_parlimen = [
    'siapakah yang mempersoalkan tindakan polis', 
    'apakah ketua polis negara yang perlukan dibuat',
    'who need to instruct chief police',
]

### Predict

```python
def predict(
    self,
    paragraph_text: str,
    question_texts: List[str],
    validate_answers: bool = True,
    validate_questions: bool = False,
    minimum_threshold_question: float = 0.05,
    **kwargs,
):
    """
    Predict extractive answers from questions given a paragraph.

    Parameters
    ----------
    paragraph_text: str
    question_texts: List[str]
        List of questions, results really depends on case sensitive questions.
    validate_answers: bool, optional (default=True)
        if True, will check the answer is inside the paragraph.
    validate_questions: bool, optional (default=True)
        if True, validate the question is subset of the paragraph using `sklearn.feature_extraction.text.CountVectorizer`
    minimum_threshold_question: float, optional (default=0.05)
        minimum score from `cosine_similarity`, only useful if `validate_questions = True`.
    **kwargs: vector arguments pass to huggingface `generate` method.
        Read more at https://huggingface.co/docs/transformers/main_classes/text_generation

    Returns
    -------
    result: List[str]
    """
```

In [10]:
model.predict(p_wikipedia, q_wikipedia)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['23', 'Menteri Pertahanan dan Menteri Pelajaran', '7 Januari 2004']

In [11]:
model_flan.predict(p_wikipedia, q_wikipedia)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['23', 'menggantikan bapanya duduk di kerusi Pekan', '1976']

In [12]:
model.predict(p_news, q_news)

['Bekas perdana menteri Najib Razak', 'Hilman Idham', '8 Mei']

In [13]:
model_flan.predict(p_news, q_news)

Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors


['Bekas perdana menteri Najib Razak', 'Hilman Idham', 'hari ini']

In [14]:
model.predict(p_parlimen, q_parlimen, max_length=50, 
    num_beams=5, 
    early_stopping=True)

['Menteri Dalam Negeri',
 'menjaga supaya selama Penggal Dewan Rakyat Parlimen',
 'Menteri Dalam Negeri']

In [15]:
model_flan.predict(p_parlimen, q_parlimen, max_length=50, 
    num_beams=5, 
    early_stopping=True)

['Tuan Yang diPertua', 'Setiausaha Dewan Rakyat', 'Majlis']

### Able to mixed MS and EN

Paragraph or questions can be both MS or EN or mixed.

In [16]:
# https://ms.wikipedia.org/wiki/Mohd_Najib_bin_Abdul_Razak

p_wikipedia = """
Najib razak telah dipilih untuk Parlimen Malaysia pada tahun 1976, 
pada usia 23 tahun, menggantikan bapanya duduk di kerusi Pekan yang berpangkalan di Pahang. 
Dari tahun 1982 hingga 1986 beliau menjadi Menteri Besar (Ketua Menteri) Pahang, 
sebelum memasuki persekutuan Kabinet Tun Dr Mahathir Mohamad pada tahun 1986 sebagai Menteri Kebudayaan, Belia dan Sukan. 
Beliau telah berkhidmat dalam pelbagai jawatan Kabinet sepanjang baki tahun 1980-an dan 1990-an, termasuk sebagai Menteri Pertahanan dan Menteri Pelajaran. 
Beliau menjadi Timbalan Perdana Menteri pada 7 Januari 2004, berkhidmat di bawah Perdana Menteri Tun Dato' Seri Abdullah Ahmad Badawi, 
sebelum menggantikan Badawi setahun selepas Barisan Nasional mengalami kerugian besar dalam pilihan raya 2008. 
Di bawah kepimpinan beliau, Barisan Nasional memenangi pilihan raya 2013, 
walaupun buat kali pertama dalam sejarah Malaysia pembangkang memenangi majoriti undi popular.
"""
q_wikipedia = [
    'pada umur berapa najib razak menggantikan ayahnya', 
    'Apakah jawatan yang pernah dipegang oleh Najib Razak',
    'what age najib razak replaced his father']

In [17]:
model.predict(p_wikipedia, q_wikipedia)

['23', 'Menteri Pertahanan dan Menteri Pelajaran', '23']

In [18]:
model_flan.predict(p_wikipedia, q_wikipedia)

['23', 'menggantikan bapanya duduk di kerusi Pekan', '23']

In [19]:
# https://www.malaysiakini.com/news/574914

p_news = """
Bekas perdana menteri Najib Razak mempersoalkan tindakan polis yang menurutnya tidak serta-merta mengeluarkan kenyataan berhubung dakwaan Adun Perikatan Nasional (PN) "merancang" insiden rogol.
Sedangkan, kata ahli parlimen Pekan itu, polis pantas mengeluarkan kenyataan apabila dia dilapor terlupa mengimbas MySejahtera sebelum masuk restoran.
"Berita Najib lupa scan MySejahtera tular, kenyataan polis terus keluar. Berita Dr Mahathir Mohamad lupa scan, kenyataan, polis serta-merta keluar.
"Sebab itu saya pelik kenapa pihak polis belum sempat keluar apa-apa kenyataan berhubung kes seorang gadis membuat laporan polis untuk dakwa Adun PN rancang insiden rogolnya," katanya di Facebook hari ini.
Najib merujuk dakwaan seorang wanita yang mendakwa dirogol kenalan kepada Adun Gombak Setia, Hilman Idham.
Wanita itu mendakwa ahli politik dari Bersatu berkenaan merancang insiden yang berlaku pada 5 Dis lalu.
Menurut laporan polis pada 8 Mei, mangsa mendakwa kejadian itu berlaku di sebuah hotel di Selangor, yang pada masa itu berada di bawah perintah kawalan pergerakan bersyarat (PKPB).
"""

q_news = [
    'siapakah yang mempersoalkan tindakan polis', 
    'siapakah Adun Gombak Setia',
    'who is the victim on 8 may incident',
    'where does the 8 may incident happened',
]

In [20]:
model.predict(p_news, q_news)

['Bekas perdana menteri Najib Razak',
 'Hilman Idham',
 'Hilman Idham',
 'hotel di Selangor']

In [21]:
model_flan.predict(p_news, q_news)

['Bekas perdana menteri Najib Razak',
 'Hilman Idham',
 'Hilman Idham',
 'sebuah hotel di Selangor']

In [22]:
# https://en.wikipedia.org/wiki/Anwar_Ibrahim

p_anwar = """
A graduate from University of Malaya, Anwar started his political career as one of the founders of youth organisation Angkatan Belia Islam Malaysia (ABIM).[4] After joining the United Malays National Organisation (UMNO), the major party in the long-time ruling BN coalition, Anwar held several cabinet positions in successive governments in the 1980s and 1990s. He was Deputy Prime Minister and Finance Minister during the 1990s and was prominent in Malaysia's response to the 1997 Asian financial crisis.[5] In 1998, he was removed from all posts by Prime Minister Mahathir Mohamad and went on to spearhead the Reformasi movement against the government. Anwar was jailed in April 1999 after a trial for sodomy and corruption that was criticised by human rights groups[6] and several foreign governments, until his release in 2004 after his conviction was overturned. He made a comeback as Leader of the Opposition from 2008 to 2015 and coalesced opposition parties into the Pakatan Rakyat (PR) coalition, which unsuccessfully contested in the 2008 and 2013 general elections. He disputed the results of the 2013 elections and led a protest in response.

In 2014, Anwar's attempt to become Selangor head of government in the 2014 Kajang Move led to a nine-month political crisis, which ended when he was sentenced to another five years imprisonment after a second sodomy conviction in 2015.[7] While still in prison, Anwar re-joined Mahathir Mohamad in the new Pakatan Harapan (PH) coalition in absentia, which went on to win the 2018 general election. Mahathir outlined a plan for Anwar to take over from himself as Prime Minister after an unspecified interim period.[8] Anwar received a royal pardon from Yang di-Pertuan Agong Muhammad V[9] and was released from prison in May 2018.[10][11] He returned to parliament in the 2018 Port Dickson by-election while his wife Wan Azizah Wan Ismail served as the Deputy Prime Minister in the PH administration. The collapse of the coalition during 2020–22 Malaysian political crisis led to the new Perikatan Nasional (PN) coalition under Muhyiddin Yassin being sworn in and Anwar becoming Leader of the Opposition for the second time in May 2020.
"""

q_anwar = [
    'tahun bila kajang move', 
    'macam mana anwar mula his political career',
    'siapa yang bagi pengampunan kepada anwar',
]

In [23]:
model.predict(p_anwar, q_anwar)

['2014',
 'youth organisation Angkatan Belia Islam Malaysia',
 'Yang di-Pertuan Agong Muhammad V']

In [24]:
model_flan.predict(p_anwar, q_anwar)

['2014',
 'A graduate from University of Malaya',
 'Anwar was jailed in April 1999 after a trial for sodomy and corruption']